# OroraTech Challenge

#### Importing Packages

In [2]:
import netCDF4 as nc
import os

import numpy as np
from matplotlib import pyplot as plt
from mpl_toolkits.basemap import Basemap
import numpy as np
import cv2 as cv
import geopandas as gpd
import pandas as pd
from shapely.geometry import Point
from keplergl import KeplerGl

#### Fetching the Data Set

In [3]:
folder_path = 'input_data'
radiance_filename = 'VNP02MOD_NRT.A2020233.1000.001.nc'
geolocation_filename = 'VNP03MOD_NRT.A2020233.1000.001.nc'

# Creating the File Path
radiance_filepath = os.path.join(folder_path, radiance_filename)
geolocation_filepath = os.path.join(folder_path, geolocation_filename)

# Fetching the data file and storing in a variable.
radiance_netcdf = nc.Dataset(radiance_filepath)
geolocation_netcdf = nc.Dataset(geolocation_filepath)

In [4]:
# Fetching the Variables from the respective group of the nc file.
m13_fire_variable = radiance_netcdf['/observation_data/M13']
latitude_variable = geolocation_netcdf['/geolocation_data/latitude']
longitude_variable = geolocation_netcdf['/geolocation_data/longitude']

# Fetching the units and the array of values.
m13_units = m13_fire_variable.units
m13_fire_array = m13_fire_variable[:]

# Fetching the array of values.
latitude_array = latitude_variable[:]
longitude_array = longitude_variable[:]

In [12]:
# Rescaling the Fire Array to display on the screen
def rescaleFrame(frame, scale):
    width = int(frame.shape[1] * scale)
    height = int(frame.shape[0] * scale)
    dimensions = (width, height)

    return cv.resize(frame, dimensions, interpolation=cv.INTER_AREA)

m13_fire_array_rescaled = rescaleFrame(m13_fire_array, 0.25)

# Applying a binary threshold value.
ret, thresh1 = cv.threshold(m13_fire_array_rescaled, 0.85,1, cv.THRESH_BINARY)

# Plotting the Threshold image on the screen.
cv.imshow('Threshold-Fire',thresh1)

# Saving the image in output directory.
frame_normed = 255 * (thresh1 - thresh1.min()) / (thresh1.max() - thresh1.min())
frame_normed = np.array(frame_normed, np.int)
output_folder = 'output'
threshold_filename = 'binarythreshold_image.png'
threshold_filepath = os.path.join(output_folder, threshold_filename)
cv.imwrite(filename=threshold_filepath, img=frame_normed)
cv.waitKey(0)

C:\TheFourthReich\anaconda3\envs\gisenv\lib\site-packages\ipykernel_launcher.py:19: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations


-1

### Steps for moving the data into DataFrame (& GeoDataFrame)

In [ ]:
# Flattening the array and converting them to a list to insert them in a dataframe.
m13_fire_list = list(m13_fire_array.flatten())
longitude_list = list(longitude_array.flatten())
latitude_list = list(latitude_array.flatten())

